In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import os
from scipy.stats import linregress
from pprint import pprint

In [2]:
data_file = os.path.join("resources", "Decade Demographics.csv")
demog_df = pd.read_csv(data_file, thousands=',')
demog_df.head(10)

,Label (Grouping),United States!!Total!!Estimate,United States!!Foreign born; Entered 2010 or later!!Estimate,United States!!Foreign born; Entered 2000 to 2009!!Estimate,United States!!Foreign born; Entered before 2000!!Estimate
0,Foreign-born population,"44,932,901","11,142,149","11,035,400","22,755,352"
1,CITIZENSHIP,NaN,NaN,NaN,NaN
2,Naturalized citizen,51.6%,13.7%,47.6%,72.1%
3,Not a citizen,48.4%,86.3%,52.4%,27.9%
4,WORLD REGION OF BIRTH OF FOREIGN BORN,NaN,NaN,NaN,NaN
5,Foreign-born population excluding popu...,"44,932,799","11,142,149","11,035,400","22,755,250"
6,Europe,10.4%,8.0%,7.8%,12.8%
7,Asia,31.4%,38.8%,29.3%,28.8%
8,Africa,5.5%,9.1%,6.7%,3.2%
9,Oceania,0.7%,0.9%,0.7%,0.5%


In [3]:
#Format data frame, "or later" = 2020
demog_df = demog_df.dropna()
demog_df = demog_df.rename(columns = {"United States!!Total!!Estimate": "United States Total",
                                    "United States!!Foreign born; Entered 2010 or later!!Estimate": "2010 or later",
                                   "United States!!Foreign born; Entered 2000 to 2009!!Estimate": "2000 to 2009",
                                   "United States!!Foreign born; Entered before 2000!!Estimate": "Before 2000"})
demog_df = demog_df.reset_index(drop = True)

In [4]:
#Dropping the total column from each DataFrame Created
demog_df = demog_df.drop(["United States Total"], axis=1)

In [5]:
demog_df.head(50)

,Label (Grouping),2010 or later,2000 to 2009,Before 2000
0,Foreign-born population,"11,142,149","11,035,400","22,755,352"
1,Naturalized citizen,13.7%,47.6%,72.1%
2,Not a citizen,86.3%,52.4%,27.9%
3,Foreign-born population excluding popu...,"11,142,149","11,035,400","22,755,250"
4,Europe,8.0%,7.8%,12.8%
5,Asia,38.8%,29.3%,28.8%
6,Africa,9.1%,6.7%,3.2%
7,Oceania,0.9%,0.7%,0.5%
8,Latin America,41.5%,54.3%,52.6%
9,Northern America,1.7%,1.2%,2.1%


In [6]:
#Get total population numbers
total_pop = demog_df.iloc[:1]

#Format data frame
total_pop = total_pop.set_index("Label (Grouping)")
total_pop = total_pop.apply(lambda x: x.str.replace(",", "").astype(int))


#Get total population numbers for education
total_edu_pop = demog_df.loc[[41,46],:]

#Format data frame
total_edu_pop = total_edu_pop.set_index("Label (Grouping)")
total_edu_pop = total_edu_pop.apply(lambda x: x.str.replace(",", "").astype(int))


#Get total marital status number
total_marital_pop = demog_df.loc[[36],:]

#Format data frame
total_marital_pop = total_marital_pop.set_index("Label (Grouping)")
total_marital_pop = total_marital_pop.apply(lambda x: x.str.replace(",", "").astype(int))

In [7]:
#Get age information
immigrant_age = demog_df.iloc[12:20, :]
median_age = demog_df.iloc[21:22]

#Get gender information
immigrant_gender = demog_df.iloc[10:12, :]

#Get continents information
immigrant_continents = demog_df.iloc[4:9, :]

#Get race information
immigrant_race = demog_df.iloc[[22,29], :]

#Get one race information
immigrant_one_race = demog_df.iloc[23:28, :]

#Get 2+ race information
immigrant_more_than_one_race = demog_df.iloc[30:32, :]

#Get Marital Status information
immigrant_marital_status = demog_df.iloc[37:41, :]

#Get age 3+ in school information
immigrant_in_school = demog_df.iloc[42:46, :]

#Get 25+ education information
immigrant_edu_plus25 = demog_df.iloc[47:50, :]

In [8]:
#Set index for all DataFrames to Label groupings
immigrant_age = immigrant_age.set_index("Label (Grouping)")
median_age = median_age.set_index("Label (Grouping)")
immigrant_gender = immigrant_gender.set_index("Label (Grouping)")
immigrant_continents = immigrant_continents.set_index("Label (Grouping)")
immigrant_race = immigrant_race.set_index("Label (Grouping)")
immigrant_one_race = immigrant_one_race.set_index("Label (Grouping)")
immigrant_more_than_one_race = immigrant_more_than_one_race.set_index("Label (Grouping)")
immigrant_marital_status = immigrant_marital_status.set_index("Label (Grouping)")
immigrant_in_school = immigrant_in_school.set_index("Label (Grouping)")
immigrant_edu_plus25 = immigrant_edu_plus25.set_index("Label (Grouping)")

In [9]:
#Turn (X) to 0 in the immigrant_age DataFrame
immigrant_age = immigrant_age.replace("(X)", "00")

In [10]:
#Remove Percentage Signs, converting to float, and divide by 100 for each DataFrame created
for i in range (0, 3):
    
    #Remove "%"
    immigrant_age.iloc[:, i] = list(map(lambda x: x[:-1], immigrant_age.iloc[:, i].values))
    immigrant_gender.iloc[:, i] = list(map(lambda x: x[:-1], immigrant_gender.iloc[:, i].values))
    immigrant_continents.iloc[:, i] = list(map(lambda x: x[:-1], immigrant_continents.iloc[:, i].values))
    immigrant_race.iloc[:, i] = list(map(lambda x: x[:-1], immigrant_race.iloc[:, i].values))
    immigrant_one_race.iloc[:, i] = list(map(lambda x: x[:-1], immigrant_one_race.iloc[:, i].values))
    immigrant_more_than_one_race.iloc[:, i] = list(map(lambda x: x[:-1], immigrant_more_than_one_race.iloc[:, i].values))
    immigrant_marital_status.iloc[:, i] = list(map(lambda x: x[:-1], immigrant_marital_status.iloc[:, i].values))
    immigrant_in_school.iloc[:, i] = list(map(lambda x: x[:-1], immigrant_in_school.iloc[:, i].values))
    immigrant_edu_plus25.iloc[:, i] = list(map(lambda x: x[:-1], immigrant_edu_plus25.iloc[:, i].values))
    
    #Convert to float
    immigrant_age.iloc[:, i] = list(map(lambda x: float(x), immigrant_age.iloc[:, i].values))
    immigrant_gender.iloc[:, i] = list(map(lambda x: float(x), immigrant_gender.iloc[:, i].values))
    immigrant_continents.iloc[:, i] = list(map(lambda x: float(x), immigrant_continents.iloc[:, i].values))
    immigrant_race.iloc[:, i] = list(map(lambda x: float(x), immigrant_race.iloc[:, i].values))
    immigrant_one_race.iloc[:, i] = list(map(lambda x: float(x), immigrant_one_race.iloc[:, i].values))
    immigrant_more_than_one_race.iloc[:, i] = list(map(lambda x: float(x), immigrant_more_than_one_race.iloc[:, i].values))
    immigrant_marital_status.iloc[:, i] = list(map(lambda x: float(x), immigrant_marital_status.iloc[:, i].values))
    immigrant_in_school.iloc[:, i] = list(map(lambda x: float(x), immigrant_in_school.iloc[:, i].values))
    immigrant_edu_plus25.iloc[:, i] = list(map(lambda x: float(x), immigrant_edu_plus25.iloc[:, i].values))
    
    #Divide by 100
    immigrant_age.iloc[:, i] = list(map(lambda x: x / 100, immigrant_age.iloc[:, i].values))
    immigrant_gender.iloc[:, i] = list(map(lambda x: x / 100, immigrant_gender.iloc[:, i].values))
    immigrant_continents.iloc[:, i] = list(map(lambda x: x / 100, immigrant_continents.iloc[:, i].values))
    immigrant_race.iloc[:, i] = list(map(lambda x: x / 100, immigrant_race.iloc[:, i].values))
    immigrant_one_race.iloc[:, i] = list(map(lambda x: x / 100, immigrant_one_race.iloc[:, i].values))
    immigrant_more_than_one_race.iloc[:, i] = list(map(lambda x: x / 100, immigrant_more_than_one_race.iloc[:, i].values))
    immigrant_marital_status.iloc[:, i] = list(map(lambda x: x / 100, immigrant_marital_status.iloc[:, i].values))
    immigrant_in_school.iloc[:, i] = list(map(lambda x: x / 100, immigrant_in_school.iloc[:, i].values))
    immigrant_edu_plus25.iloc[:, i] = list(map(lambda x: x / 100, immigrant_edu_plus25.iloc[:, i].values))


In [11]:
# Convert to actual population numbers, round to the nearest whole person
imm_age_numbers = round(immigrant_age.multiply(total_pop.iloc[0], axis='columns')).astype(int)
imm_gender_numbers = round(immigrant_gender.multiply(total_pop.iloc[0], axis='columns')).astype(int)
imm_continents_numbers = round(immigrant_continents.multiply(total_pop.iloc[0], axis='columns')).astype(int)
imm_race_numbers = round(immigrant_race.multiply(total_pop.iloc[0], axis='columns')).astype(int)
imm_one_race_numbers = round(immigrant_one_race.multiply(total_pop.iloc[0], axis='columns')).astype(int)
imm_more_than_one_race_numbers = round(immigrant_more_than_one_race.multiply(total_pop.iloc[0], axis='columns')).astype(int)
imm_marital_status_numbers = round(immigrant_marital_status.multiply(total_marital_pop.iloc[0], axis='columns')).astype(int)
imm_in_school_numbers = round(immigrant_in_school.multiply(total_edu_pop.iloc[0], axis='columns')).astype(int)
imm_edu_plus25_numbers = round(immigrant_edu_plus25.multiply(total_edu_pop.iloc[0], axis='columns')).astype(int)

In [12]:
# Removing the spaces before each index
imm_age_numbers.index = imm_age_numbers.index.str.strip()
imm_gender_numbers.index = imm_gender_numbers.index.str.strip()
imm_continents_numbers.index = imm_continents_numbers.index.str.strip()
imm_race_numbers.index = imm_race_numbers.index.str.strip()
imm_one_race_numbers.index = imm_one_race_numbers.index.str.strip()
imm_more_than_one_race_numbers.index = imm_more_than_one_race_numbers.index.str.strip()
imm_marital_status_numbers.index = imm_marital_status_numbers.index.str.strip()
imm_in_school_numbers.index = imm_in_school_numbers.index.str.strip()
imm_edu_plus25_numbers.index = imm_edu_plus25_numbers.index.str.strip()

In [13]:
# Exporting DataFrames to CSV's
path = os.path.join("Outputs", "Age.csv")
imm_age_numbers.to_csv(path)

path = os.path.join("Outputs", "Gender.csv")
imm_gender_numbers.to_csv(path)

path = os.path.join("Outputs", "Continents.csv")
imm_continents_numbers.to_csv(path)

path = os.path.join("Outputs", "Race.csv")
imm_race_numbers.to_csv(path)

path = os.path.join("Outputs", "One Race.csv")
imm_one_race_numbers.to_csv(path)

path = os.path.join("Outputs", "More Than One Race.csv")
imm_more_than_one_race_numbers.to_csv(path)

path = os.path.join("Outputs", "Marital Status.csv")
imm_marital_status_numbers.to_csv(path)

path = os.path.join("Outputs", "In School.csv")
imm_in_school_numbers.to_csv(path)

path = os.path.join("Outputs", "Education 25+.csv")
imm_edu_plus25_numbers.to_csv(path)